### **Data Preprocess**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix

In [ ]:
# iit = itemset_item_training
iit = pd.read_csv("/content/drive/MyDrive/datamining_dataset/itemset_item_training.csv",names = ['itemsets','item'])

In [ ]:
grouped_df=iit.groupby('itemsets').agg(list)
df = grouped_df.reset_index()

# Define a lambda function to extract values from the lists
extract_values = lambda row, col_num: row['item'][col_num] if col_num < len(row['item']) else 'nan'

# Apply the lambda function to create new columns
max_num_items = max([len(item_list) for item_list in df['item']])
for i in range(max_num_items):
    df[f'item{i+1}'] = df.apply(lambda row: extract_values(row, i), axis=1)

# Drop the original 'item' column
df.drop('item', axis=1, inplace=True)

# Print the transformed DataFrame
print(df)


       itemsets  item1  item2  item3  item4  item5
0             0  10176  21460  22555  31044    nan
1             2  17798  33815   3002    nan    nan
2             3  13117  12117  35243    nan    nan
3             6  25031   6932  25215  37817    nan
4             7  17648  16915  27897  14190    nan
...         ...    ...    ...    ...    ...    ...
22151     27688  35681   3620   6285  30200  15499
22152     27689  21449  19120   1112   9376  33644
22153     27690  12140   1951  39405  30280    nan
22154     27691   5903  22917  39122   7768    nan
22155     27693  22757  41249  33768   5600    nan

[22156 rows x 6 columns]


In [ ]:
grouped_df=iit.groupby('itemsets').agg(list)
df = grouped_df.reset_index()
# Reset index to convert col1 from an index to a regular column
grouped_df = grouped_df.reset_index()

# Concatenate col2 values into a single string column
grouped_df['item'] = grouped_df['item'].apply(lambda x: ' '.join(map(str, x)))
print(grouped_df)

data = list(grouped_df["item"].apply(lambda x:x.split(" ") ))
data

       itemsets                         item
0             0      10176 21460 22555 31044
1             2             17798 33815 3002
2             3            13117 12117 35243
3             6       25031 6932 25215 37817
4             7      17648 16915 27897 14190
...         ...                          ...
22151     27688  35681 3620 6285 30200 15499
22152     27689  21449 19120 1112 9376 33644
22153     27690       12140 1951 39405 30280
22154     27691        5903 22917 39122 7768
22155     27693       22757 41249 33768 5600

[22156 rows x 2 columns]


[['10176', '21460', '22555', '31044'],
 ['17798', '33815', '3002'],
 ['13117', '12117', '35243'],
 ['25031', '6932', '25215', '37817'],
 ['17648', '16915', '27897', '14190'],
 ['24295', '5068', '34247', '11789'],
 ['20815', '7021', '40969', '22391'],
 ['5843', '22271', '20915'],
 ['38694', '4091', '3957', '33111', '41154'],
 ...]

In [ ]:
#Let's transform the list, with one-hot encoding
from mlxtend.preprocessing import TransactionEncoder
a = TransactionEncoder()
a_data = a.fit(data).transform(data)
df = pd.DataFrame(a_data,columns=a.columns_)
df = df.replace(False,0)
df

,0,1,10,100,1000,10000,10001,10002,10003,10004,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22151,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22152,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22153,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22154,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### **Association Rule Mining (Apriori Algorithm)**

In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules
#set a threshold value for the support value and calculate the support value.
df = apriori(df, min_support = 0.2, use_colnames = True)
df

In [ ]:
#Let's view our interpretation values using the Associan rule function.
df_ar = association_rules(df, metric = "confidence", min_threshold = 0.6)
df_ar